In [35]:
import mxnet as mx
from mxnet import nd, autograd, gluon,init
import matplotlib.pyplot as plt
from mxnet.gluon import nn
import time

In [6]:
data_ctx  = mx.gpu()
model_ctx = mx.gpu()

In [8]:
import pandas as pd

In [17]:
bucket='sagemaker-hepmxnet'
train_data_key = 'train_data.csv'
test_data_key = 'test_data.csv'
val_data_key = 'val_data.csv'

train_loc = 's3://{}/{}'.format(bucket, train_data_key)
test_loc = 's3://{}/{}'.format(bucket, test_data_key)
val_loc = 's3://{}/{}'.format(bucket, val_data_key)

train_df = pd.read_csv(train_loc)
test_df = pd.read_csv(test_loc)
val_df = pd.read_csv(val_loc)

In [18]:
train_arr = nd.array(train_df)
test_arr  = nd.array(test_df)
val_arr   = nd.array(val_df)

In [27]:
## neural net architecture
net = nn.Sequential()
net.add(
    nn.Dense(9,activation='relu'),
    nn.BatchNorm(),
    nn.Dropout(0.7),
    
    nn.Dense(512,activation='relu'),
    nn.BatchNorm(),
    nn.Dropout(0.7),
    
    nn.Dense(512,activation='relu'),
    nn.BatchNorm(),
    nn.Dropout(0.7),
    
    nn.Dense(512,activation='relu'),
    nn.BatchNorm(),
    nn.Dropout(0.7),
    
    nn.Dense(1,activation='sigmoid')
)
net.initialize(init=init.Xavier())

In [28]:
## Loss and optimizer
binary_cross_entropy = gluon.loss.SigmoidBinaryCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'Adam', {'learning_rate': 0.001})

In [33]:
## hyper parameters
batch_size = 256
training_epoch = 50

In [29]:
## ACC calculator
def acc(output, label):
    # output: (batch, num_output) float32 ndarray
    # label: (batch, ) int32 ndarray
    return (output.argmax(axis=1) ==
            label.astype('float32')).mean().asscalar()

In [36]:
for epoch in range(training_epoch):
    train_loss, train_acc, valid_acc = 0., 0., 0.
    tic = time.time()
    for data,label in train_arr:
        # forward + backward
        with autograd.record():
            output = net(data)
            loss = binary_cross_entropy(output, label)
        loss.backward()
        # update parameters
        trainer.step(batch_size)
        # calculate training metrics
        train_loss += loss.mean().asscalar()
        train_acc += acc(output, label)
    # calculate validation accuracy
    for data, label in valid_data:
        valid_acc += acc(net(data), label)
    print("Epoch %d: loss %.3f, train acc %.3f, test acc %.3f, in %.1f sec" % (
            epoch, train_loss/len(train_data), train_acc/len(train_data),
            valid_acc/len(valid_data), time.time()-tic))

ValueError: too many values to unpack